<figure>
  <IMG SRC="https://upload.wikimedia.org/wikipedia/commons/thumb/d/d5/Fachhochschule_Südwestfalen_20xx_logo.svg/320px-Fachhochschule_Südwestfalen_20xx_logo.svg.png" WIDTH=250 ALIGN="right">
</figure>

# Skriptsprachen
### Sommersemester 2021
Prof. Dr. Heiner Giefers

## Sprache von Texten erkennen

In dieser Aufgabe wollen wir uns mit dem Problem beschäftigen, die Sprache zu bestimmen, in der ein gewisser Text verfasst ist.
Es gibt verschiedenste Ansätze, dieses Problem zu lösen.
Wir werden ein sehr einfaches, Daten-getriebenes Verfahren einsetzen, bei dem die charakteristische *Buchstabenhäufigkeit* einer Sprache verwendet wird.

Die Idee ist folgende: In den verschiedenen Sprachen kommen bestimmte Buchstaben - jedenfalls gefühlt - unterschiedlich häufig vor.
Im Englischen, zum Beispiel, ist das *y* deutlich häufige als vor als im Deutschen, im Französischen ist das *e* (ohne Akzent) warscheinlich seltener als in anderen Sprachen.
Versuchen wir also herauszufinden, ob diese Idee funktioniert.

Um die mittleren Buschstabenhäufigkeiten in den verschiedenen Sprachen zu ermitteln, brauchen wir zunächst einmal **Daten**.
Eine recht gute Quelle für Texte in verschiedenen Sprachen, ist [Wikipedia](https://www.wikipedia.org).
Von dort werden wir verschiedene, zufällige Seiten in den unterschiedlichen Sprachen aufrufen und den Textinhalt dieser Seiten herunterladen.
Für den Zugriff auf Wikipedia Seiten empfliehlt sich die Installation des Python-Pakets *Wikipedia-API*.
Falls nicht vorhanden, installieren wir dieses Paket direkt aus dem Notebook heraus.

In [1]:
import sys
!{sys.executable} -m pip install wikipedia-api

In der nächsten Code-Zelle werden die Methoden `random_page` und `random_text` implementiert.
`random_text(size,lang)` liefert einen Text mit `size` Zeichen in der Sprache `lang`.
Um zufällige Wikipedia-Seiten herunterzuladen wir die Methode `random_page` verwendet.
Wikipedia bietet über die URL [wikipedia.org/wiki/Special:Random](http://wikipedia.org/wiki/Special:Random) die Möglichkeit, eine zufällige Seite auszuwählen.
Über diese URL holen wir uns den Titel der zufälligen Seite und laden den Inhalt schließlich mit der Wikipedia-API herunter.

In [ ]:
import re
import requests
from bs4 import BeautifulSoup
import wikipediaapi

def random_page(wiki, lang="de"): 
  r = requests.get("https://" + lang + ".wikipedia.org/wiki/Special:Random")
  soup = BeautifulSoup(r.content)
  # Schneitet 12 Buchstaben vom Ende ab
  # Funktioniert nicht bei allen Sprachen
  #page = wiki.page(soup.title.text[:-12])
  # Such nach "Wikip" im Titel
  # Nimm den vorderen Teil ohne " - " (3 Buchstaben)
  title = soup.title.text.split('Wikip')[0]
  page = wiki.page(title[:-3])
  assert page.exists(), f"Die Seite {page} in {lang} ibt es nicht"
  return page.text

def random_text(size,lang="de"): 
  WIKI = wikipediaapi.Wikipedia(
        language=lang,
        extract_format=wikipediaapi.ExtractFormat.WIKI
  )

  text = ''
  while(len(text)<size):
    t = random_page(WIKI, lang)
    t_cleaned = re.sub('[^a-zA-Z ]', '', t)
    #Alles zu Kleinbuchstaben
    t_cleaned = t_cleaned.lower()
    text += t_cleaned

  return text[:size]


**Aufgabe:** Verwenden Sie die Funktion `random_text` um einen Englischen Text mit 200 Zeichen zu erzeugen. Teilen Sie den Text in einzelne Worte auf, die in der Liste `worte` abgelegt sind. Die `for`-Schleife durchläuft diese Liste und gibt die einzelnen Worte aus.

In [ ]:
worte = None
# YOUR CODE HERE
raise NotImplementedError()

for i, wort in enumerate(worte):
    print(f"{i}) {wort}", end=' ')

**Aufgabe:** Als nächstes wollen wir die Häufigkeiten der Buchstaben im Text bestimmen.
Dazu verwenden wir ein *Dictionary* `buchstaben`, das zu Anfang leer ist und gehen folgendermaßen vor:
1. Wir durchlaufen alle Buchstaben des Textes.
2. Prüfe, ob der aktuelle Buchstabe als *key* im Dictionary vorkommt
    1. Falls er nicht vorkommt, fügen wir den Buchstaben als Schlüssel hinzu und setzen den Wert des Schlüssels auf 1.
    2. Falls er vorkommt, erhöhen wir den Wert des Eintrags um 1
3. Nachdem alle Buchstaben des Textes verarbeitet sind, durchlaufen wir alle Schlüssel des Dictionaries `buchstaben` und teilen jeden Wert durch die Anzahl der Buchstaben im Text. Beispiel: Wenn der Text 1000 Zeichen hat und der Wert `buchstaben['e']` gleich 100 ist, so setzen wir `buchstaben['e']` auf 0.1

In [ ]:
def letter_frequency(text):
    buchstaben = {}
    # YOUR CODE HERE
    raise NotImplementedError()
    
    return buchstaben


In [ ]:
test_freq = letter_frequency("pizza")
assert 'z' in test_freq
assert 'e' not in test_freq
assert test_freq['z']==0.4

Wir haben nun eine Methode `letter_frequency`, mit der wir die relativen Häufigkeiten der Buchstaben eines Textes bestimmen können.
Um einen ersten Eindruck von diesen Häufigkeiten zu bekommen, werden wir ein Säulendiagramm erstellen.
Dazu verwenden wir die Bibliothek `matplotlib`

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

text = random_text(1000, 'de')
buchstaben = letter_frequency(text)

plt.figure(figsize=(30,5)) 

x = range(len(buchstaben))
y = list(buchstaben.values())
x_ticks = list(buchstaben.keys())
plt.bar(x, y, align='center')
plt.xticks(x, x_ticks, rotation=0)
plt.show()

Wie man sieht, sind die Einträge im Dictionary unsortiert.
Weder die Schlüssel noch die Werte folgen einer Ordnung.

Um die Buchstabenhäufigkeiten aber später vergleichen zu können, wollen wir einen *Vektor* erzeugen, bei dem Buchstaben immer an gleicher Stelle stehen. Z.B. `a` an Index 0, `b` an Index 1, usw.
Eine Idee könnte sien, die Schlüssel zu sortieren.
Hierbei ist aber Vorsicht geboten, denn es könnt sein, dass ein Buchstabe im Text nicht vorkommt.
In dem Fall, hätte der Vektor zu wenige Einträge.

Wir verfolgen daher einen anderen Weg.
Über eine `for`-Schleife zählen wir alle Buchstaben von *a* bis *z* auf, schauen nach, ob sie im Dictionary enthalten sind und übernehmen den Wert in eine Liste.
Ist der Buchstabe nicht enthalten, übernehmen wir den Wert 0.0 in die Liste.

Eine Schleife, die alle Buchstaben durchläuft könnte so aussehen
```pypthon
for i in range(ord('a'), ord('z')+1):
    print(chr(i), end=' ')
```

`ord('a')` liefert den ASCII Code des Zeichens *a*.
So kann die Schleife die ASCII Codes von *a* bis *z* durchlaufen.
Im Schleifenrumpf erzeugen wir aus dem ASCII Code mit der Typumwnadlung `chr` wieder ein Zeichen.

**Aufgabe:** Schreiben Sie eine Funktion `frequency_vector(d)`, die aus einem Dictionary `d` ein Tupel mit 26 Einträgen erzeugt, in denen jeweils die relative Häufigkeit der Buchstaben von  *a* bis *z* im Text steht.

In [ ]:
def frequency_vector(d):
    f = []
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
test_d = {'a': 0.5, 'd': 0.5}
test_l = [0.0]*26
test_l[0]=0.5
test_l[3]=0.5
assert frequency_vector(test_d)==tuple(test_l)

In der folgenden Code-Zelle werden die zuvor entwickelten Funktionen verwendet, um die relativen Häufigkeiten der Buchstaben in den Sprachen *Deutsch*, *Englisch*, *Spanisch* und *Französisch* zu bestimmen.
Wir verwenden dazu jeweils 10000 Zeichen aus Wikipedia Artikeln.
Die Resultate werden im Dictionary `haeufigkeiten` unter den Schlüsseln `'de'`, `'en'`, `'es'` und `'fr'` abgelegt.

In [ ]:
languages = ['de', 'en', 'es', 'fr']

def get_lang_freq(lang):
  text = random_text(10000, lang)
  buchstaben = letter_frequency(text)
  return frequency_vector(buchstaben)

haeufigkeiten = {}
for l in languages:
  haeufigkeiten[l] = get_lang_freq(l)

Nun haben wir verschiedene Vektoren, die wir als Maßstab für die einzelnen Sprachen verwenden können.
Als Nächstes wollen wir überprüfen, of wir diese Vektoren verwenden können, um für einen neuen Text vorherzusagen, in welcher Sprache er geschrieben ist.

Dazu definiert die folgende Code-Zelle die Funktion `get_page_freq`, mit der wir den Inhalt einer speziellen Wikipediaseite herunterladen und die relative Häufigkeit der Buschstaben in dieser Seite berechnen können.

In [ ]:
def get_wiki_page(term, lang='de'): 
  WIKI = wikipediaapi.Wikipedia(
        language=lang,
        extract_format=wikipediaapi.ExtractFormat.WIKI
  )
  page = WIKI.page(term)
  assert page.exists()
  t = page.text
  t_cleaned = re.sub('[^a-zA-Z]', '', t)
  t_cleaned = t_cleaned.lower()
  return t_cleaned

def get_page_freq(term, lang='de'):
  text = get_wiki_page(term, lang=lang)
  buchstaben = letter_frequency(text)
  return frequency_vector(buchstaben)

mypage = get_page_freq("Hochschule")

An dieser Stelle haben wir fünf Vektoren, je einen, der die Häufigkeiten der Buchstaben in den Sprachen *Deutsch*, *Englisch*, *Spanisch* und *Französisch* beschreibt sowie einen weiteren, mit den relativen Häufigkeiten der Buchstaben in *unserem* Text.
Nun wollen wir ausrechnen, zu welchem der vier Sprach-Vektoren unser Buchstaben-Vektor *am ähnlichsten* ist, bzw. zu welchem Sprach-Vektor er die wenigsten Unterschiede aufweist.

Wir werden also die einzelnen Buchstaben-Häufigkeiten in unserem Vektor mit denen im Sprach-Vektor *vergleichen*.
Mathematisch gesehen, bilden wir die Differenz der Häufigkeiten aller einzelnen Buchstaben.
Kommt z.B. in unserem Text das *e* zu 10% und im Sprach-Vektor zu 8% vor, so ist die Abweichung $0.1-0.08=0.02$ für den Buchstaben *e*.

Wenn wir nun alle Abweichungen aufsummieren, kann es zu einem Problem kommen.
Manche Differenzen werden positiv, andere negativ sein.
Eine einfache Summe wird die negativen und positiven Abweichungen gegeneinander aufrechen.

Um dieses Problem zu beheben, gibt es verschiedene Möglichkeiten.
Man kann die Absolutwerte der Abweichungen aufsummieren, oder auch deren Quadratwerte.
In jedem Fall wird die Summe am Ende durch die Anzahl der Einträge des Vektors geteilt, um den Mittleren Fehler zu berechnen.

Verwendet man zum Berechnung die Absolutwerte, nennt man das Maß [*Mittlerer absoluter Fehler*](https://de.wikipedia.org/wiki/Mittlerer_absoluter_Fehler) (oder *mean absolute error*), verwendet man die Fehlerquadrate heißt das Maß [*Mittlerer quadratischer Fehler*](https://de.wikipedia.org/wiki/Mittlere_quadratische_Abweichung) (oder *mean squared error*).

**Aufgabe:** Implementieren Sie die Funktion `mse` zur Berechnung des Mittleren Quadratischen Fehlers. 

In [ ]:
def mse(a,b):
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
a = [1,2,3,4,5]
b = [3,2,5,4,5]
assert mse(a,b)==1.6

Nun können wir unsere Vektoren vergleichen und herausfinden, zu welcher Sprache unser Vektor am ähnlichsten ist.

In [ ]:
def get_prop(mypage, languages):
  fehler = {}
  for l in languages:
    fehler[l] = mse(mypage,languages[l])

  fehlersumme = sum(fehler.values())

  props = {}
  for f in fehler:
    props[f] = 100-fehler[f]*100/fehlersumme


  prop_sum = sum(props.values())

  for f in props:
    props[f] = props[f]*100/prop_sum

  return props

p = get_prop(mypage, haeufigkeiten)
p

Das Dictionary `p` enthält die berechneten Wahrscheinlichkeiten, mit denen der Text in den jeweiligen Sprachen verfasst ist.
Man sollte sehen, dass bei einem deutschen Text, der key `'de'` den höchsten Wert enthält. 

Wie man sieht, ist die Häufigkeit der verschiedenen Buchstaben ein mögliches Vorhersagekriterium, allerdings kein sehr gutes. Die Wahrscheinlichkeiten für Deutsch und Englisch sind i.d.R. nicht sehr verschieden.

Um das Verfahren zu verbessern, könnte man statt einzelne Buchstaben ganze Worte betrachten.
Oder alternativ dazu, Sequenzen von $n$ aufeinanderfolgenden Buchstaben, den sogenannten [n-Grammen](https://de.wikipedia.org/wiki/N-Gramm).

Wenn man einen Text auf n-Gramme untersucht, betrachtet man Folgen von $n$ Buchstaben von jedem Buchstaben aus.
In der folgenden Code-Zelle verwenden wir das Wort *Waschmaschine* als Beispiel.
Das Wort hat 13 Buchstaben und besitzt damit 11 Trigramme (3-Gramme) von denen eines (das *sch*) doppelt vorkommt.

In [ ]:
wort = 'Waschmaschine'
[wort[i:i+3] for i in range(len(wort)-2)]

**Aufgabe:** 
1. Generieren Sie einen zufälligen Text mit 2000 Zeichen
2. Teilen Sie den Text so auf, dass Sie die einzelnen Worte in einer Liste ablegen. Berechnen Sie die 3-Gramme für alle Worte im Text und speichern Sie die 3-Gramme in einer Liste
3. Es sollen nur die 3-Gramme betrachtet werden. D.h, wenn ein Wort weniger als 3 Buchstaben besitzt, nehmen Sie es nicht in die Liste auf
4. Geben Sie die Anzahl aller 3-Gramme aus, die in ihrem Text enthalten sind
5. Geben Sie die Anzahl aller eindeutigen 3-Gramme aus, d.h. ohne doppelte Vorkommen

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()